# Custom substructure loading from PDB and NAGL charge assignment

This is a RED example showcasing early prototypes of functionality that we plan to make available in the coming year. We provide no guarantee of stability or correct operation, but this is an example of what may be to come.

This example will:
* Load a modified protein from PDB using `Topology.from_pdb`
* Charge the protein with NAGL

See also:
https://gist.github.com/Yoshanuikabundi/66007cb9966b1455a259baaf7cd7e7c3#file-ncaa_parametrization-ipynb

Why it's red:
- NAGL is experimental and pre-release
- Charging a whole protein with NAGL will not give the same charges as FF14SB
- Sage parameters have not been rigourously validated for use in non-canonical amino acids.

## Introduction

Here we have a PDB structure containing a protein with sequence "ACE-ALA5-DYE-ALA5-NME". The DYE residue is a cysteine labelled with a fluorophore via a maleimide "click" reaction.

In [ ]:
import nglview

w = nglview.show_structure_file("inputs/3ip9_dye_solvated.pdb")
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="backbone or DYE")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w

## Load a PDB with non-canonical amino acid

Let's get started on the workflow by making a representation of the non-canonical amino acid, which we'll use to teach our PDB loader about the new substructure. I'm labeling the connection points with [Fr] atoms.

In [ ]:
from openff.toolkit import Molecule, Topology

# Make unnatural AA as its own mol. Since a residue is a substructure, not a valid molecule,
# use something that's easy to recognize as a "cap". Here I use Fr since it makes one bond,
# we're super unlikely to see Fr anywhere else, and Sage doesn't support it, so if
# we make a mistake and it gets left behind we'll eventually get an error.
substructure_mol = Molecule.from_smiles(
    "C1=CC2=C(C=C1N3C(=O)C[C@H](SC[C@H](N[Fr])C(=O)[Fr])C3=O)C(=O)O[C@]24C5=C(C=C(C=C5)O)OC6=C4C=CC(=C6)O"
)
substructure_mol.visualize(show_all_hydrogens=False)

In [ ]:
# Label the atoms with whether they're leaving
for atom in substructure_mol.atoms:
    if atom.symbol == "Fr":
        atom.metadata["substructure_atom"] = False
    else:
        atom.metadata["substructure_atom"] = True

In [ ]:
top = Topology.from_pdb(
    "inputs/3ip9_dye_solvated.pdb",
    # _additional_substructures is a PROTOTYPE.
    # Its behavior and input type are likely to change.
    _additional_substructures=[substructure_mol],
)

In [ ]:
w = top.visualize()
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w

## Compute NAGL partial charges

Here, for simplicity, we simply compute NAGL charges for the entire peptide. A more rigorous approach might combine NAGL or Antechamber charges for the dye with Amber library charges for the other residues, as was done in a previous workshop:

https://www.youtube.com/watch?v=lZ4UUgHQpWg

In [ ]:
from openff.nagl_models import list_available_nagl_models

[path.name for path in list_available_nagl_models()]

In [ ]:
protein = top.molecule(0)

In [ ]:
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

protein.assign_partial_charges(
    "openff-gnn-am1bcc-0.1.0-rc.2.pt", toolkit_registry=NAGLToolkitWrapper()
)
print(protein.partial_charges)

### Comparing NAGL charges to Amber library charges

NAGL charges are quite different to the charges assigned by Amber, as they are based on AM1-BCC rather than RESP calculations. These methods are considered to produce "compatible" charges for intermolecular interactions, but this will have an effect on the Amber force field's intramolecular interactions. A future OpenFF protein force field is being developed with charges much more similar to NAGL, and NAGL charges will later be optimized in concert with OpenFF forcce fields.

Library charges can only be assigned to a molecule if all atoms in that molecule can be assigned one; they cannot be mixed within a molecule with other charge methods. We can define a dummy library charge for generic atoms to allow Amber library charges to be assigned to the remainder of the protein.

In [ ]:
from io import StringIO

from openff.interchange.smirnoff import SMIRNOFFElectrostaticsCollection
from openff.toolkit import ForceField

# Construct a force field that assigns any atom a charge of 0
generic_librarycharge = """<?xml version="1.0" encoding="utf-8"?>
<SMIRNOFF version="0.3" aromaticity_model="OEAroModel_MDL">        
  <LibraryCharges version="0.3">
    <LibraryCharge smirks="[*:1]" charge1="0.0 * elementary_charge" id="generic_zero"></LibraryCharge>
  </LibraryCharges>
</SMIRNOFF>
"""

# Combine the above library charge with ff14SB
# General force field first so that Amber wins any conflicts!
with StringIO(generic_librarycharge) as generic_librarycharge_offxml:
    amber_charges = ForceField(
        generic_librarycharge_offxml,
        "ff14sb_off_impropers_0.0.4.offxml",
    )

# Use the above force field to force Amber library charges on all atoms
# Construct an Electrostatics collection directly to avoid issues with
# missing bonded or vdW parameters
librarycharges_collection = SMIRNOFFElectrostaticsCollection.create(
    parameter_handler=[
        amber_charges["Electrostatics"],
        amber_charges["LibraryCharges"],
    ],
    topology=protein.to_topology(),
)

Identify the charges in parts of the protein other than the dye (where the Amber charges are undefined).

In [ ]:
from openff.units import unit

dye_atom_indices = [
    atom.molecule_atom_index
    for atom in [res for res in protein.residues if res.residue_name == "DYE"][0].atoms
]

nagl_partial_charges = [
    q.m_as(unit.elementary_charge)
    for i, q in enumerate(protein.partial_charges)
    if i not in dye_atom_indices
] * unit.elementary_charge
amber_partial_charges = [
    q.m_as(unit.elementary_charge)
    for i, q in enumerate(librarycharges_collection.charges.values())
    if i not in dye_atom_indices
] * unit.elementary_charge

Plot the NAGL charges against Amber charges.

In [ ]:
from matplotlib import pyplot

pyplot.scatter(
    nagl_partial_charges.m,
    amber_partial_charges.m,
    c=range(len(nagl_partial_charges)),
    marker=".",
)
pyplot.xlabel(f"nagl charges ({nagl_partial_charges.u})")
pyplot.ylabel(f"reference charges ({amber_partial_charges.u})")
pyplot.axline((0, 0), slope=1, linestyle=":")

If you're interested, we can break down assigned charges by atom and residue name:

In [ ]:
# # Note: amber charges vary when multiple residue types are assigned the same residue name; 
# # eg, with different protonation states or for terminal residues

# from collections import defaultdict
# from functools import partial

# # Break charges down by atom and residue name
# nagl_charges_by_atomtype = defaultdict(partial(defaultdict, list))
# ff14sb_charges_by_atomtype = defaultdict(partial(defaultdict, list))
# for nagl_charge, ff14sb_charge, atom in zip(
#     protein.partial_charges, librarycharges_collection.charges.values(), protein.atoms
# ):
#     nagl_charges_by_atomtype[atom.metadata["residue_name"]][atom.name].append(
#         nagl_charge.m_as(unit.elementary_charge)
#     )
#     ff14sb_charges_by_atomtype[atom.metadata["residue_name"]][atom.name].append(
#         ff14sb_charge.m_as(unit.elementary_charge)
#     )

# # Create a plot for the 21 different residue names (20 canonical AAs plus DYE)
# fig, axs = pyplot.subplots(
#     nrows=7,
#     ncols=3,
#     figsize=(20, 30),
#     sharey=True,
#     dpi=180,
#     constrained_layout=True,
#     gridspec_kw={"hspace": 0.1, "wspace": 0.05},
# )

# # Plot the charges from both force fields
# for (resname, by_atomname), ax in zip(nagl_charges_by_atomtype.items(), axs.flatten()):
#     atomnames, nagl_charges = zip(*by_atomname.items())
#     amber_charges = [
#         ff14sb_charges_by_atomtype[resname][atomname] for atomname in atomnames
#     ]
#     boxplot = ax.boxplot(
#         nagl_charges,
#         showfliers=False,
#         labels=atomnames,
#         medianprops={"color": "black"},
#         positions=range(len(nagl_charges)),
#     )
#     nagl_events = ax.eventplot(nagl_charges, orientation="vertical", color="orange")
#     amber_events = ax.eventplot(amber_charges, orientation="vertical")
#     ax.set_title(f"{resname} (n={len(nagl_charges[0])})")
#     ax.set_xmargin(0)

#     if resname == "MET":
#         amber_events[0].set_label("Amber charges")
#         nagl_events[0].set_label("NAGL charges")
#         boxplot["boxes"][0].set_label("NAGL charges box plot")
#         ax.legend(loc="lower right")
#         ax.set_ylabel("Partial Charge (e-)")

## Parametrize and simulate the peptide with Sage, Amber ff14sb, and NAGL charges

In [ ]:
from openff.toolkit import ForceField

sage_ff14sb = ForceField("openff-2.2.0.offxml", "ff14sb_off_impropers_0.0.4.offxml")

In [ ]:
ic = sage_ff14sb.create_interchange(top, charge_from_molecules=[protein])

In [ ]:
import openmm

simulation = ic.to_openmm_simulation(
    openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtoseconds,
    )
)

# Energy minimize
simulation.minimizeEnergy()

# Add a reporter to record the structure every 1000 steps
dcd_reporter = openmm.app.DCDReporter("ncaa.dcd", 1000)
simulation.reporters.append(dcd_reporter)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1 * openmm.unit.minute)

In [ ]:
from utils import nglview_show_openmm

w = nglview_show_openmm(simulation.topology, "ncaa.dcd")
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w